In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from PIL import Image
import os
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms

# Define a transformation to preprocess the images
transform = transforms.Compose([
    transforms.Grayscale(),  # Convert the image to grayscale (assuming your images are colored)
    transforms.Resize((8, 8)),  # Resize the image to 8x8
    transforms.ToTensor()  # Convert the image to a PyTorch tensor
])

# Load and preprocess your images
image_paths = ["Q5_1.png", "Q5_2.png", "Q5_3.png"]
images = []

for path in image_paths:
    # Load the image using PIL
    img = Image.open(os.path.join(path))
    # Apply the transformation
    img_tensor = transform(img)
    #convert to binary
    img_tensor = torch.where(img_tensor[0] > 0.5, torch.tensor(1.0), torch.tensor(0.0))
    # Add the preprocessed image tensor to the list
    images.append(img_tensor)

# Create a batch tensor by stacking the image tensors
batch_tensor = torch.stack(images)

#show one image
print(batch_tensor[0])
plt.imshow(batch_tensor[0].squeeze(), cmap='gray')
plt.show()


# define labels :  ص -> 0-> 001     گ-<1->010        ب->2->100
labels = torch.tensor([0, 1, 2])
# Split the dataset into training and testing sets
x_train, x_test, y_train, y_test = batch_tensor, batch_tensor, labels, labels

# Convert data to PyTorch tensors
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).view(-1, 64)  # Flatten 8x8 images to 1D tensors
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32).view(-1, 64)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create a custom MLP model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Initialize the model, loss function, and optimizer
input_size = 64  # 8x8 images flattened
hidden_size = 128  # Number of neurons in the hidden layer
num_classes = 3  # Three classes for three alphabets (assuming 3 classes)

model = MLP(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create DataLoader for training and testing datasets
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=3, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=3, shuffle=False)

# Training the model
num_epochs = 100
for epoch in range(num_epochs):
    total_loss = 0
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}')

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on test set: {accuracy:.2f}%')


In [ ]:
# !pip install Pillow torch